In [ ]:
import pandas as pd
import numpy as np
import copy

dqn_c = "models/DQN_crypto_81_06_17_24_ca8svpca/evaluation_metrics_for_anova"
ppo_c = "models/PPO_crypto_83_06_17_24_gzsbryxw/evaluation_metrics_for_anova"
pcn_c = "models/PCN_sweep_sp_81_tlhm6t79_tlhm6t79/evaluation_metrics_for_anova"

paths = [pcn_c, dqn_c, ppo_c]
output = []

for p in paths:
    main_path = p
    results_df = pd.DataFrame(columns=['Episode return', 'Maximal drawdown', 'Volatility', 'Sharpe Ratio', 'Calmar Ratio', 'Ulcer index', 'Transaction costs'])

    for i in range(30):
        path = f"{main_path}{i}.csv"
        df = pd.read_csv(path)

        df['portfolio return'] = df['portfolio value'].pct_change().fillna(0)
        initial_investment = df.loc[0, 'portfolio value']
        df['cumulative portfolio return'] = (1 + df['portfolio return']).cumprod()
        # generate metrics
        df['daily_return'] = df['portfolio value'].pct_change()
        # Final Return
        final_return = df['portfolio value'].iloc[-1] / df['portfolio value'].iloc[0] - 1
        # Maximal Drawdown
        cumulative_return = (1 + df['daily_return']).cumprod()
        rolling_max = cumulative_return.cummax()
        drawdown = cumulative_return / rolling_max - 1
        max_drawdown = drawdown.min()
        # Average Volatility (annualized)
        avg_volatility = df['daily_return'].std() * np.sqrt(252)  # 252 trading days in a year
        # Sharpe Ratio
        risk_free_rate = 0.01  # Assuming a risk-free rate of 1%
        excess_return = df['daily_return'].mean() - (risk_free_rate / 252)
        sharpe_ratio = excess_return / df['daily_return'].std() * np.sqrt(252)
        # Calmar Ratio
        calmar_ratio = excess_return / abs(max_drawdown)
        # Ulcer index
        drawdown_squared = drawdown**2
        ulcer_index = np.sqrt(np.mean(drawdown_squared))

        transaction_costs = df['transaction costs'].sum()/initial_investment

        results_df.loc[i] = [final_return, max_drawdown, avg_volatility, sharpe_ratio, calmar_ratio, ulcer_index, transaction_costs]
    output.append(copy.deepcopy(results_df))

lengths = ["PCN", "DQN", "PPO"]
for i, l in enumerate(lengths):
    output[i]['model'] = l
df1, df2, df3 = output
df_combined = pd.concat([df1, df2, df3])
df_combined.groupby('model').mean()


In [ ]:


from scipy.stats import f_oneway
# Example for final return
f_stat, p_val = f_oneway(df1['Episode return'], df2['Episode return'], df3['Episode return'])
print(f'ANOVA result for Episode return: F-statistic={f_stat}, p-value={p_val}')
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import pandas as pd

# Combine the data into one DataFrame for Tukey's HSD
combined_data = pd.concat([df1.assign(Strategy='PCN'),
                           df2.assign(Strategy='DQN'),
                           df3.assign(Strategy='PPO')])

# Perform Tukey's HSD test
tukey = pairwise_tukeyhsd(endog=combined_data['Episode return'], groups=combined_data['Strategy'], alpha=0.005)

In [ ]:

ep_30 = "models/PCN_sweep_sp_17_9s2d0yaa_9s2d0yaa/evaluation_metrics_front.csv"
ep_100 = "models/PCN_sweep_sp_17_y73vig0d_y73vig0d/evaluation_metrics_front.csv"
ep_252 = 'models/PCN_sweep_sp_17_vxj8pdbw_vxj8pdbw/evaluation_metrics_front.csv'
ep_30 = "models/PCN_sweep_sp_17_9s2d0yaa_9s2d0yaa/evaluation_metrics_backtest.csv"
ep_100 = "models/PCN_sweep_sp_17_y73vig0d_y73vig0d/evaluation_metrics_backtest.csv"
ep_252 = 'models/PCN_sweep_sp_17_vxj8pdbw_vxj8pdbw/evaluation_metrics_backtest.csv'
ep_30 = "models/PCN_sweep_sp_17_9s2d0yaa_9s2d0yaa/evaluation_metrics_front.csv"
ep_100 = "models/PCN_sweep_sp_17_y73vig0d_y73vig0d/evaluation_metrics_front.csv"
ep_252 = 'models/PCN_sweep_sp_17_vxj8pdbw_vxj8pdbw/evaluation_metrics_front.csv'
# ep_30 = "models/PCN_sweep_sp_17_9s2d0yaa_9s2d0yaa/evaluation_metrics_backtest_validation.csv"
# ep_100 = "models/PCN_sweep_sp_17_y73vig0d_y73vig0d/evaluation_metrics_backtest_validation.csv"
# ep_252 = 'models/PCN_sweep_sp_17_vxj8pdbw_vxj8pdbw/evaluation_metrics_backtest_validation.csv'
df = pd.read_csv(ep_30)
df2 = pd.read_csv(ep_100)
df3 = pd.read_csv(ep_252)


stocks = ['AAPL', 'MSFT', 'JNJ', 'PG', 'TSLA', 'NFLX', 'KO', 'V']
stocks = [x+" price" for x in stocks]
returns = []
for stock in stocks:
    returns = [f"{stock}_return"]
    df[f"{stock}_return"] = df[stock].pct_change().fillna(0)

df["market_return"] = df[returns].mean(axis=1)
initial_investment = df.loc[0, 'portfolio value']
df['cumulative portfolio return'] = (1 + df['market_return']).cumprod()
# Calculate the portfolio value over time
df['market value'] = initial_investment * df['cumulative portfolio return']
# df['portfolio volatility'] = abs(df['risk reward'])

date_range = pd.date_range(start='2020-08-24', periods=len(df), freq='B')
plt.figure(figsize=(14, 7))
plt.plot(date_range, df['portfolio value'], label='30-days episode based strategy')
plt.plot(date_range, df2['portfolio value'], label='100-days episode based strategy')
plt.plot(date_range, df3['portfolio value'], label='252-days episode based strategy')
plt.plot(date_range, df['market value'], label='Baseline', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.title('Portfolio value for different episode length configurations')
plt.legend()
plt.grid(True)
plt.show()
print(df['transaction costs'].sum())
print(df2['transaction costs'].sum())
print(df3['transaction costs'].sum())
print(df2['actions'].value_counts())
print(df['AAPL price'].tail())


In [ ]:
df["pct"] = df["portfolio value"].pct_change().fillna(0)
df2["pct"] = df2["portfolio value"].pct_change().fillna(0)
df3["pct"] = df3["portfolio value"].pct_change().fillna(0)
from scipy.stats import f_oneway

# Example for final return
f_stat, p_val = f_oneway(df['pct'], df2['pct'], df3['pct'])
print(f'ANOVA result for pct returns: F-statistic={f_stat}, p-value={p_val}')
import scikit_posthocs as sp

# Example for final return using the Kruskal-Wallis test
posthoc = sp.posthoc_dunn([df['pct'], df2['pct'], df3['pct']], p_adjust='bonferroni')
print(posthoc)


In [ ]:
import pandas as pd
import numpy as np
import copy


ep_30 = "models/PCN_sweep_sp_17_9s2d0yaa_9s2d0yaa/evaluation_metrics_for_anova_side"
ep_100 = "models/PCN_sweep_sp_17_y73vig0d_y73vig0d/evaluation_metrics_for_anova_side"
ep_252 = 'models/PCN_sweep_sp_17_vxj8pdbw_vxj8pdbw/evaluation_metrics_for_anova_side'

paths = [ep_30, ep_100, ep_252]
output = []

for p in paths:
    main_path = p
    results_df = pd.DataFrame(columns=['Final Return', 'Maximal Drawdown', 'Volatility', 'Sharpe Ratio', 'Calmar Ratio', 'Ulcer index', 'Transaction costs'])

    for i in range(30):
        path = f"{main_path}{i}.csv"
        df = pd.read_csv(path)
        stocks = ['AAPL', 'MSFT', 'JNJ', 'PG', 'TSLA', 'NFLX', 'KO', 'V']
        stocks = [x+" price" for x in stocks]
        returns = []
        for stock in stocks:
            returns = [f"{stock}_return"]
            df[f"{stock}_return"] = df[stock].pct_change().fillna(0)
        df["market_return"] = df[returns].mean(axis=1)
        initial_investment = df.loc[0, 'portfolio value']
        df['cumulative portfolio return'] = (1 + df['market_return']).cumprod()
        # Calculate the portfolio value over time
        df['market value'] = initial_investment * df['cumulative portfolio return']
        # generate metrics
        df['daily_return'] = df['market value'].pct_change()
        # Final Return
        final_return = df['market value'].iloc[-1] / df['market value'].iloc[0] - 1
        # Maximal Drawdown
        cumulative_return = (1 + df['daily_return']).cumprod()
        rolling_max = cumulative_return.cummax()
        drawdown = cumulative_return / rolling_max - 1
        max_drawdown = drawdown.min()
        # Average Volatility (annualized)
        avg_volatility = df['daily_return'].std() * np.sqrt(252)  # 252 trading days in a year
        # Sharpe Ratio
        risk_free_rate = 0.01  # Assuming a risk-free rate of 1%
        excess_return = df['daily_return'].mean() - (risk_free_rate / 252)
        sharpe_ratio = excess_return / df['daily_return'].std() * np.sqrt(252)
        # Calmar Ratio
        calmar_ratio = excess_return / abs(max_drawdown)
        # Ulcer index
        drawdown_squared = drawdown**2
        ulcer_index = np.sqrt(np.mean(drawdown_squared))

        transaction_costs = df['transaction costs'].sum()/initial_investment

        results_df.loc[i] = [final_return, max_drawdown, avg_volatility, sharpe_ratio, calmar_ratio, ulcer_index, transaction_costs]

    #transaction_costs/=initial_investment
    output.append(copy.deepcopy(results_df))

lengths = [30, 100, 252]
for i, l in enumerate(lengths):
    output[i]['episode length'] = l
df1, df2, df3 = output
df_combined = pd.concat([df1, df2, df3])
df_combined.groupby('episode length').mean()


In [ ]:
import scipy.stats as stats
correlation, p_value = stats.pearsonr(df_combined['Final Return'], df_combined['episode length'])
print("Pearson correlation coefficient:", correlation)
print("P-value:", p_value)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


sp_PCN_100_best = "models/PCN_sweep_sp_57_xo7t4bkx_xo7t4bkx/evaluation_metrics.csv"


# PCN 100 new obs - does not change actions
new_obs_sp = "models/PCN_sweep_sp_17_oivusehp_oivusehp/evaluation_metrics.csv"
new_obs_sp_extended = "models/PCN_sweep_sp_17_v0244b2c_v0244b2c/evaluation_metrics.csv"
dqn_100 = "models/DQN_sp_extended_17_06_15_24_d8hah8u1/evaluation_metrics.csv"
ppo_100 = "models/PPO_sp_extended_81_06_15_24_pcr0nder/evaluation_metrics.csv"



path = new_obs_sp_extended

df = pd.read_csv(path)

stocks = ['AAPL',
        'MSFT',
        'JNJ',
        'PG',
        'TSLA',
        'NFLX',
        'KO',
        'V',
        'GOOGL',
        'AMZN',
        'META',
        'JPM',
        'UNH',
        'HD',
        'VZ',
        'DIS',
        'NVDA',
        'MA',
        'ADBE',
        'IBM']
stocks = [x+" price" for x in stocks]
returns = []
for stock in stocks:
    returns = [f"{stock}_return"]
    df[f"{stock}_return"] = df[stock].pct_change().fillna(0)

df["market_return"] = df[returns].mean(axis=1)
initial_investment = df.loc[0, 'portfolio value']
df['cumulative market return'] = (1 + df['market_return']).cumprod()
# Calculate the market value over time
df['market value'] = initial_investment * df['cumulative market return']
# df['portfolio volatility'] = abs(df['risk reward'])
df.to_csv(path[:-4]+"_processed.csv")

path = ppo_100

df1 = pd.read_csv(path)

returns = []
for stock in stocks:
    returns = [f"{stock}_return"]
    df1[f"{stock}_return"] = df1[stock].pct_change().fillna(0)

df1["market_return"] = df1[returns].mean(axis=1)
initial_investment = df.loc[0, 'portfolio value']
df1['cumulative market return'] = (1 + df1['market_return']).cumprod()
# Calculate the market value over time
df1['market value'] = initial_investment * df1['cumulative market return']
# df['portfolio volatility'] = abs(df['risk reward'])
df1.to_csv(path[:-4]+"_1_processed.csv")
path = dqn_100
df2 = pd.read_csv(path)

returns = []
for stock in stocks:
    returns = [f"{stock}_return"]
    df2[f"{stock}_return"] = df2[stock].pct_change().fillna(0)

df2["market_return"] = df2[returns].mean(axis=1)
initial_investment = df.loc[0, 'portfolio value']
df2['cumulative market return'] = (1 + df2['market_return']).cumprod()
# Calculate the market value over time
df2['market value'] = initial_investment * df2['cumulative market return']
# df['portfolio volatility'] = abs(df['risk reward'])
df2.to_csv(path[:-4]+"_2_processed.csv")
date_range = pd.date_range(start='2022-04-04', periods=len(df), freq='D')
plt.figure(figsize=(14, 7))
plt.plot(date_range, df['portfolio value'], label='PCN')
plt.plot(date_range, df1['portfolio value'], label='PPO')
plt.plot(date_range, df2['portfolio value'], label='DQN')
plt.plot(date_range, df['market value'], label='Baseline', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.title('Portfolio value for S&P500 selected 20 stocks scenario')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# back tests results on validation data
pcn = "models/PCN_sweep_sp_17_v0244b2c_v0244b2c/evaluation_metrics_backtest_validation.csv"
dqn = "models/DQN_sp_extended_17_06_15_24_d8hah8u1/evaluation_metrics_backtest_validation.csv"
ppo = "models/PPO_sp_extended_81_06_15_24_pcr0nder/evaluation_metrics_backtest_validation.csv"
df = pd.read_csv(pcn)
df2 = pd.read_csv(dqn)
df3 = pd.read_csv(ppo)


stocks = ['AAPL',
        'MSFT',
        'JNJ',
        'PG',
        'TSLA',
        'NFLX',
        'KO',
        'V',
        'GOOGL',
        'AMZN',
        'META',
        'JPM',
        'UNH',
        'HD',
        'VZ',
        'DIS',
        'NVDA',
        'MA',
        'ADBE',
        'IBM']
stocks = [x+" price" for x in stocks]
returns = []
for stock in stocks:
    returns = [f"{stock}_return"]
    df[f"{stock}_return"] = df[stock].pct_change().fillna(0)

df["market_return"] = df[returns].mean(axis=1)
initial_investment = df.loc[0, 'portfolio value']
df['cumulative portfolio return'] = (1 + df['market_return']).cumprod()
# Calculate the portfolio value over time
df['market value'] = initial_investment * df['cumulative portfolio return']
# df['portfolio volatility'] = abs(df['risk reward'])

date_range = pd.date_range(start='2020-08-24', periods=len(df), freq='B')
plt.figure(figsize=(14, 7))
plt.plot(date_range, df['portfolio value'], label='PCN')
plt.plot(date_range, df2['portfolio value']/10, label='DQN')
plt.plot(date_range, df3['portfolio value']/10, label='PPO')
plt.plot(date_range, df['market value'], label='Baseline', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.title('Portfolio value for different episode length configurations')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import copy

pcn = "models/PCN_sweep_sp_17_v0244b2c_v0244b2c/evaluation_metrics_backtest_validation.csv"
dqn = "models/DQN_sp_extended_17_06_15_24_d8hah8u1/evaluation_metrics_backtest_validation.csv"
ppo = "models/PPO_sp_extended_81_06_15_24_pcr0nder/evaluation_metrics_backtest_validation.csv"

stocks = ['AAPL',
        'MSFT',
        'JNJ',
        'PG',
        'TSLA',
        'NFLX',
        'KO',
        'V',
        'GOOGL',
        'AMZN',
        'META',
        'JPM',
        'UNH',
        'HD',
        'VZ',
        'DIS',
        'NVDA',
        'MA',
        'ADBE',
        'IBM']
for i in range(len(stocks)):
    stocks[i] += " price"
paths = [pcn, dqn, ppo]
dfs = [pd.read_csv(path) for path in paths]
df_baselines = copy.deepcopy(dfs[0])
output = []
returns = []
for stock in stocks:
    returns = [f"{stock}_return"]
    df_baselines[f"{stock}_return"] = df_baselines[stock].pct_change().fillna(0)

df_baselines["market_return"] = df_baselines[returns].mean(axis=1)
initial_investment = df.loc[0, 'portfolio value']
df_baselines['cumulative market return'] = (1 + df_baselines['market_return']).cumprod()
# Calculate the market value over time
df_baselines['market value'] = initial_investment * df_baselines['cumulative market return']
df_baselines['portfolio value'] = df_baselines['market value']

dfs.append(df_baselines)
results_df = pd.DataFrame(columns=['Episode return', 'Maximal drawdown', 'Volatility', 'Sharpe Ratio', 'Calmar Ratio', 'Ulcer index', 'Transaction costs'])

for df in dfs:

    df['portfolio return'] = df['portfolio value'].pct_change().fillna(0)
    initial_investment = df.loc[0, 'portfolio value']
    df['cumulative portfolio return'] = (1 + df['portfolio return']).cumprod()
    # Calculate the portfolio value over time
    #market_return = df['market value'].iloc[-1] / df['market value'].iloc[0] - 1
    # generate metrics
    df['daily_return'] = df['portfolio value'].pct_change()
    # Final Return
    final_return = df['portfolio value'].iloc[-1] / df['portfolio value'].iloc[0] - 1
    # Maximal Drawdown
    cumulative_return = (1 + df['daily_return']).cumprod()
    rolling_max = cumulative_return.cummax()
    drawdown = cumulative_return / rolling_max - 1
    max_drawdown = drawdown.min()
    # Average Volatility (annualized)
    avg_volatility = df['daily_return'].std() * np.sqrt(252)  # 252 trading days in a year
    # Sharpe Ratio
    risk_free_rate = 0.01  # Assuming a risk-free rate of 1%
    excess_return = df['daily_return'].mean() - (risk_free_rate / 252)
    sharpe_ratio = excess_return / df['daily_return'].std() * np.sqrt(252)
    # Calmar Ratio
    calmar_ratio = excess_return / abs(max_drawdown)
    # Ulcer index
    drawdown_squared = drawdown**2
    ulcer_index = np.sqrt(np.mean(drawdown_squared))

    transaction_costs = df['transaction costs'].sum()/initial_investment

    results_df.loc[i] = [final_return, max_drawdown, avg_volatility, sharpe_ratio, calmar_ratio, ulcer_index, transaction_costs]
    output.append(copy.deepcopy(results_df))

lengths = ["PCN", "DQN", "PPO", "Baseline"]
for i, l in enumerate(lengths):
    output[i]['model'] = l
df1, df2, df3, df4 = output
df_combined = pd.concat([df1, df2, df3, df4])
df_combined.groupby('model').mean()


In [ ]:
import pandas as pd
import numpy as np
import copy

dqn_c = "models/DQN_crypto_81_06_17_24_ca8svpca/evaluation_metrics_backtest_validation.csv"
ppo_c = "models/PPO_crypto_83_06_17_24_gzsbryxw/evaluation_metrics_backtest_validation.csv"
pcn_c = "models/PCN_sweep_sp_81_tlhm6t79_tlhm6t79/evaluation_metrics_backtest_validation.csv"
df = pd.read_csv(dqn_c)
df2 = pd.read_csv(ppo_c)
df3 = pd.read_csv(pcn_c)


stocks = ["BTC-USD", "ETH-USD", "LTC-USD", "XRP-USD", "XMR-USD", 
           "DASH-USD", "ETC-USD", "ZEC-USD", "DCR-USD", "WAVES-USD"]
stocks = [x+" price" for x in stocks]

paths = [pcn_c, dqn_c, ppo_c]
dfs = [pd.read_csv(path) for path in paths]
df_baselines = copy.deepcopy(dfs[0])
output = []
returns = []
for stock in stocks:
    returns = [f"{stock}_return"]
    df_baselines[f"{stock}_return"] = df_baselines[stock].pct_change().fillna(0)

df_baselines["market_return"] = df_baselines["BTC-USD price_return"]
initial_investment = df.loc[0, 'portfolio value']
df_baselines['cumulative market return'] = (1 + df_baselines['market_return']).cumprod()
# Calculate the market value over time
df_baselines['market value'] = initial_investment * df_baselines['cumulative market return']
df_baselines['portfolio value'] = df_baselines['market value']

dfs.append(df_baselines)
results_df = pd.DataFrame(columns=['Episode return', 'Maximal drawdown', 'Volatility', 'Sharpe Ratio', 'Calmar Ratio', 'Ulcer index', 'Transaction costs'])

for df in dfs:
    # returns = []
    # for stock in stocks:
    #     returns = [f"{stock}_return"]
    #     df[f"{stock}_return"] = df[stock].pct_change().fillna(0)

    # df["market_return"] = df[returns].mean(axis=1)
    # initial_investment = df.loc[0, 'portfolio value']
    # df['cumulative market return'] = (1 + df['market_return']).cumprod()
    # # Calculate the market value over time
    # df['market value'] = initial_investment * df['cumulative market return']

    df['portfolio return'] = df['portfolio value'].pct_change().fillna(0)
    initial_investment = df.loc[0, 'portfolio value']
    df['cumulative portfolio return'] = (1 + df['portfolio return']).cumprod()
    # Calculate the portfolio value over time
    #market_return = df['market value'].iloc[-1] / df['market value'].iloc[0] - 1
    # generate metrics
    df['daily_return'] = df['portfolio value'].pct_change()
    # Final Return
    final_return = df['portfolio value'].iloc[-1] / df['portfolio value'].iloc[0] - 1
    # Maximal Drawdown
    cumulative_return = (1 + df['daily_return']).cumprod()
    rolling_max = cumulative_return.cummax()
    drawdown = cumulative_return / rolling_max - 1
    max_drawdown = drawdown.min()
    # Average Volatility (annualized)
    avg_volatility = df['daily_return'].std() * np.sqrt(252)  # 252 trading days in a year
    # Sharpe Ratio
    risk_free_rate = 0.01  # Assuming a risk-free rate of 1%
    excess_return = df['daily_return'].mean() - (risk_free_rate / 252)
    sharpe_ratio = excess_return / df['daily_return'].std() * np.sqrt(252)
    # Calmar Ratio
    calmar_ratio = excess_return / abs(max_drawdown)
    # Ulcer index
    drawdown_squared = drawdown**2
    ulcer_index = np.sqrt(np.mean(drawdown_squared))

    transaction_costs = df['transaction costs'].sum()/initial_investment

    results_df.loc[i] = [final_return, max_drawdown, avg_volatility, sharpe_ratio, calmar_ratio, ulcer_index, transaction_costs]
    output.append(copy.deepcopy(results_df))

lengths = ["PCN", "DQN", "PPO", "Baseline"]
for i, l in enumerate(lengths):
    output[i]['model'] = l
df1, df2, df3, df4 = output
df_combined = pd.concat([df1, df2, df3, df4])
df_combined.groupby('model').mean()
